## Analyse Binette results

Let's visualize the results from Binette and compare them to the initial bin sets used as input. 

### Import Necessary Libraries

First, we'll need to import the necessary libraries for our analysis and plotting:

In [1]:
import pandas as pd
from pathlib import Path
import plotly.express as px

# This is needed to properly display Plotly graphs in the documentation
import plotly.io as pio
pio.renderers.default = "sphinx_gallery"

### Load Binette Results

Now, let's load the final Binette quality report into a Pandas DataFrame:

In [2]:
binette_result_file = "./binette_results/final_bins_quality_reports.tsv"
df_binette = pd.read_csv(binette_result_file, sep='\t')
df_binette['tool'] = "binette"  # Add a column to label the tool
df_binette['index'] = df_binette.index  # Add an index column
df_binette

,bin_id,origin,name,completeness,contamination,score,size,N50,contig_count,tool,index
0,17075,diff,44 - 10,100.00,0.05,99.90,4672665,82084,93,binette,0
1,39427,diff,36 - 6,99.90,0.20,99.50,2796605,41151,98,binette,1
2,47060,union,58 | 33,98.59,0.83,96.93,4601336,41016,165,binette,2
3,47177,union,91 | 25 | 55,96.10,0.34,95.42,2598718,11891,312,binette,3
4,21248,diff,65 - 8 - 28,91.98,1.71,88.56,1768095,9976,250,binette,4
5,44137,diff,76 - 13 - 28,92.63,2.41,87.81,3726254,5669,850,binette,5
6,31703,diff,31 - 7 - 61,81.73,0.84,80.05,1665233,8518,248,binette,6
7,13475,diff,47 - 37,72.89,2.39,68.11,1241829,5061,252,binette,7
8,47926,union,75 | 30,74.31,4.26,65.79,3293949,2954,1262,binette,8
9,46775,union,42 | 102,62.94,2.75,57.44,1293571,3783,419,binette,9


### Load and Combine Input Bin Quality Reports

Next, we will load the quality reports of the input bin sets, computed by various tools and saved by Binette. We’ll combine these into a single DataFrame and add a column to indicate high-quality bins. We define a high-quality bin as one with contamination ≤ 5% and completeness ≥ 90%.

In [3]:
from pathlib import Path

input_bins_quality_reports_dir = Path("binette_results/input_bins_quality_reports/")

# Initialize the list with Binette results
df_input_bin_list = [df_binette]

# Load each input bin quality report
for input_bin_metric_file in input_bins_quality_reports_dir.glob("*tsv"):
    tool = input_bin_metric_file.name.split('.')[1].split('_')[0]  # Extract tool name from file name
    df_input = pd.read_csv(input_bin_metric_file, sep='\t')
    df_input['index'] = df_input.index
    df_input['tool'] = tool
    df_input_bin_list.append(df_input)

# Combine all DataFrames into one
df_bins = pd.concat(df_input_bin_list)

# Add a column to indicate high-quality bins
df_bins["High quality bin"] = (df_bins['completeness'] >= 90) & (df_bins['contamination'] <= 5)

# Display relevant columns
df_bins[[ "tool", "completeness", "contamination", "size", "N50", "contig_count"]]


,tool,completeness,contamination,size,N50,contig_count
0,binette,100.00,0.05,4672665,82084,93
1,binette,99.90,0.20,2796605,41151,98
2,binette,98.59,0.83,4601336,41016,165
3,binette,96.10,0.34,2598718,11891,312
4,binette,91.98,1.71,1768095,9976,250
...,...,...,...,...,...,...
20,semibin2,8.28,0.01,358822,3296,106
21,semibin2,8.12,0.02,353499,3949,90
22,semibin2,7.74,0.01,351540,4284,85
23,semibin2,6.18,0.00,250833,3607,66


### Plot bin completeness and contamination
With the DataFrame containing both Binette’s final bins and the input bins, we can now create a scatter plot to visualize the results:

In [4]:
import plotly.express as px

# Create a scatter plot to visualize completeness and contamination
fig = px.scatter(df_bins, 
                 x="completeness", 
                 y="contamination", 
                 color="High quality bin", 
                 size="size",  
                 facet_row="tool",
                 title="Bin Quality Comparison",
                )

# Update layout for better visibility
fig.update_layout(
    width=600,
    height=800,
    legend_title="High Quality Bin",
    title="Comparison of Bin Quality Metrics"
)

# Show the plot
fig.show()

We can see that binette bins are the one displaying the most high quality bins (completeness ≥ 90% and contamination ≤ 5%).




### Comparing Binning Tools Using Bin Score Curves

A common way to compare bin sets is by sorting the bins based on their scores and plotting them against their index.

Here’s how we can create such a plot:

In [5]:
# Calculate the score for each bin
df_bins['completeness - 2*contamination'] = df_bins['completeness'] - 2 * df_bins['contamination']

# Plot the score against the bin index
fig = px.line(df_bins, x="index", y='completeness - 2*contamination', color="tool", markers=True)
fig.update_layout(width=600, height=500)
fig.show()

From the plot, you might notice that Concoct has a lot of bins with lower quality scores. Let’s zoom in to get a better look:

In [6]:
# Adjust the plot view to zoom in
fig.update_layout(
    xaxis_range=[-1, 20],  # Zoom on x-axis
    yaxis_range=[0, 100],  # Zoom on y-axis
    width=600,
    height=500
)
fig.show()

Binette line consistently appears above the other binning tools. This indicates that Binette produce higher-quality bins compared to the initial bin sets.

### Plot Number of High-Quality Bins per Bin Set

Let's plot the number of bins falling into different quality categories. We’ll focus on bins with a maximum of 10% contamination and classify them into three completeness categories:

- **`> 50% and ≤ 70%`**
- **`> 70% and ≤ 90%`**
- **`> 90%`**

First, let’s group and count the bins in each category:

In [7]:
# Define the contamination cutoff
contamination_cutoff = 10

# Create filters for completeness categories
low_contamination_filt = df_bins['contamination'] <= contamination_cutoff
high_completeness_filt = df_bins['completeness'] > 90
medium_completeness_filt = df_bins['completeness'] > 70
low_completeness_filt = df_bins['completeness'] > 50

# Define quality categories
quality  = f'Contamination ≤ {contamination_cutoff} and<br>Completeness'
df_bins.loc[low_contamination_filt & low_completeness_filt, quality] =  '> 50% and ≤ 70%'
df_bins.loc[low_contamination_filt & medium_completeness_filt, quality] =  '> 70% and ≤ 90%'
df_bins.loc[low_contamination_filt & high_completeness_filt, quality] = '> 90%'

# Group and count bins by quality category and tool
df_bins_quality_grouped = df_bins.groupby([quality, 'tool']).agg(bin_count=('bin_id', 'count')).reset_index()
df_bins_quality_grouped

,Contamination ≤ 10 and<br>Completeness,tool,bin_count
0,> 50% and ≤ 70%,binette,5
1,> 50% and ≤ 70%,maxbin2,1
2,> 50% and ≤ 70%,metabat2,1
3,> 50% and ≤ 70%,semibin2,2
4,> 70% and ≤ 90%,binette,3
5,> 70% and ≤ 90%,concoct,2
6,> 70% and ≤ 90%,metabat2,5
7,> 70% and ≤ 90%,semibin2,4
8,> 90%,binette,6
9,> 90%,concoct,4


Now, let’s create a bar plot to visualize the number of bins in each quality category for each bin sets:

In [8]:
# Define colors for each completeness category
color_discrete_map = {
    "> 90%": px.colors.qualitative.Prism[4],
    "> 70% and ≤ 90%": px.colors.qualitative.Prism[2],
    "> 50% and ≤ 70%": px.colors.qualitative.Prism[6]
}

# Create the bar plot
fig = px.bar(
    df_bins_quality_grouped, 
    x='tool', 
    y="bin_count", 
    color=quality,
    barmode='stack', 
    color_discrete_map=color_discrete_map, 
    text="bin_count",
    category_orders={"tool": ["binette", "semibin2", "concoct", "metabat2", "maxbin2"]},
    opacity=0.9
)

# Update layout for better appearance
fig.update_layout(
    width=600,
    height=500,
    legend=dict(traceorder="reversed")
)

fig.show()

From the plot, you can see that Binette produces more high-quality bins compared to the initial bin sets! 🎉